In [1]:
import pandas as pd
import numpy as np
import joblib
import shap
import matplotlib.pyplot as plt
import lightgbm as lgb
from config import MODELS_DIR


C:\Users\abdul\AppData\Roaming\Python\Python314\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load validation data
X_val = joblib.load(MODELS_DIR / "X_val.pkl")
y_val = joblib.load(MODELS_DIR / "y_val.pkl")



In [3]:
# Load base models
logreg = joblib.load(MODELS_DIR / "logreg_baseline.pkl")
xgb = joblib.load(MODELS_DIR / "xgb_fraud_model_tuned.pkl")
lgb_model = lgb.Booster(model_file=str(MODELS_DIR / "lgb_fraud_model_tuned.txt"))



In [4]:
# Rebuild meta-features for validation
meta_val = pd.DataFrame({
    "logreg_prob": logreg.predict_proba(X_val)[:, 1],
    "xgb_prob": xgb.predict_proba(X_val)[:, 1],
    "lgb_prob": lgb_model.predict(X_val)
})



In [5]:
# Load stacking meta-model
meta_model = joblib.load(MODELS_DIR / "stacking_meta_model.pkl")



In [6]:
# Predict probabilities
val_probs = meta_model.predict_proba(meta_val)[:, 1]